In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [2]:
# Config

BATCH_SIZE = 32
IMG_SIZE = 224
EPOCHS_PHASE_A = 10   
EPOCHS_PHASE_B = 5    
LR_PHASE_A = 1e-3
LR_PHASE_B = 1e-4
DATA_ROOT = r"D:\dataset_split"
N_LAST_LAYERS = 20  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [3]:
# Data

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root = f"{DATA_ROOT}/train", transform = train_transform)
val_dataset   = datasets.ImageFolder(root = f"{DATA_ROOT}/val", transform = val_transform)
test_dataset  = datasets.ImageFolder(root = f"{DATA_ROOT}/test", transform = val_transform)

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader   = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False)
test_loader  = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)


Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [4]:
# Model: ResNet50

resnet = models.resnet50(pretrained=True)

# Replace FC layer
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
model = resnet.to(device)

criterion = nn.CrossEntropyLoss()


# Training function

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs, phase_name = "Phase"):
    for epoch in range(epochs):
        # Train
        model.train()
        running_loss, running_correct = 0.0, 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            running_correct += (outputs.argmax(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_correct / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * imgs.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        print(f"{phase_name} Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
              f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    return model


# Phase A: Train classifier

for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.fc.parameters(), lr = LR_PHASE_A)
model = train_model(model, criterion, optimizer, train_loader, val_loader, 
                    epochs = EPOCHS_PHASE_A, phase_name = "Phase A")


# Phase B: Fine-tuning backbone

# Freeze all first
for name, param in model.named_parameters():
    param.requires_grad = False

# Unfreeze last N layers + FC
for name, param in list(model.named_parameters())[-N_LAST_LAYERS:]:
    param.requires_grad = True

# Optimizer (only train unfrozen layers)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = LR_PHASE_B)

# Train Phase B
model = train_model(model, criterion, optimizer, train_loader, val_loader, 
                    epochs = EPOCHS_PHASE_B, phase_name = "Phase B")


d:\Anaconda\envs\plant\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\plant\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Phase A Epoch [1/10] Train Loss: 0.8282 | Train Acc: 0.7624 | Val Loss: 0.4253 | Val Acc: 0.8766
Phase A Epoch [2/10] Train Loss: 0.4363 | Train Acc: 0.8647 | Val Loss: 0.3722 | Val Acc: 0.8780
Phase A Epoch [3/10] Train Loss: 0.3681 | Train Acc: 0.8845 | Val Loss: 0.3119 | Val Acc: 0.8994
Phase A Epoch [4/10] Train Loss: 0.3384 | Train Acc: 0.8886 | Val Loss: 0.2582 | Val Acc: 0.9159
Phase A Epoch [5/10] Train Loss: 0.3043 | Train Acc: 0.9011 | Val Loss: 0.2744 | Val Acc: 0.9018
Phase A Epoch [6/10] Train Loss: 0.2847 | Train Acc: 0.9040 | Val Loss: 0.2523 | Val Acc: 0.9101
Phase A Epoch [7/10] Train Loss: 0.2844 | Train Acc: 0.9056 | Val Loss: 0.2187 | Val Acc: 0.9252
Phase A Epoch [8/10] Train Loss: 0.2764 | Train Acc: 0.9066 | Val Loss: 0.2018 | Val Acc: 0.9329
Phase A Epoch [9/10] Train Loss: 0.2547 | Train Acc: 0.9152 | Val Loss: 0.2162 | Val Acc: 0.9247
Phase A Epoch [10/10] Train Loss: 0.2608 | Train Acc: 0.9115 | Val Loss: 0.2055 | Val Acc: 0.9315
Phase B Epoch [1/5] Train Los

In [5]:
# Test

model.eval()
test_loss, test_correct = 0.0, 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * imgs.size(0)
        test_correct += (outputs.argmax(1) == labels).sum().item()

test_loss /= len(test_loader.dataset)
test_acc = test_correct / len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")


# Save

torch.save(model.state_dict(), "D:/saved_models/resnet50_finetuned.pt")

Test Loss: 0.0603 | Test Acc: 0.9764
